# Ejercicio 2 - Mejoras y Comparación de Modelos de Clasificación

Este ejercicio considera completar acciones para mejorar el rendimiento de modelos de clasificación supervisada, (similar al ejercicio 1), pero se enfoca en realizar un análisis comparativo entre diferentes modelos utilizados para entender las ventajas/desventajas de unos y otros sobre este dataset y sus condiciones.

## Contexto: Análisis de éxito en campaña de marketing

Fuente: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

El foco está en la implementación de varios clasificadores para predecir el valor de un atributo de interés, desde un *dataset* de información de un resultados de personas contactadas por una campaña de marketing y que compraron la oferta (atributo "OK"), con cerca de 41.200 registros de personas contactadas.

Este conjunto de datos (abierto para este tipo de usos instruccionales), consiste en 20 atributos y 1 clase de etiquetas (totalizando 21 columnas) y corresponde a los datos de una campaña telefónica a diversos clientes en Portugal, ofreciéndoles la compra de un producto bancario. En varios casos, un cliente fue contactado varias veces antes de aceptar el depósito a plazo ofrecido por la campaña (OK = yes).

Algunos de los atributos relevantes son (combinando atributos categóricos, con numéricos):
* **Datos personales**: Edad, Ocupación, Estado Civil, Nivel de Educación.
* **Datos financieros**: Su casa tiene crédito hipotecario, default: si el crédito ha caído en quiebra; tiene un crédito de consumo.
* **Datos de contactos de la campaña actual**: Tipo de Comunicación (celular o teléfono fijo); Mes del último contacto; Día de la semana del contacto; duración de la llamada (segundos); Contacto: N° de contactos durante la campaña; DíasAtrás: días transcurridos desde último contacto; Resultado: resultado de la última llamada (falló, no-existe, éxito)
* **Datos socioeconómicos**: EmpTasaVar: tasa de variación de empleabilidad; IPC: índice de precios consumidor mensual; ICC: índice de confianza consumidor mensual; Euribor3m: tasa euribor de 3 meses indicador diario; NumEmpleados: cantidad de gente empleada, en indicador trimestral.

Esta adaptación en particular, por el equipo de R:Solver (RSolver.com), enfrenta diferentes objetivos de aprendizaje dentro de los cursos de Big Data y Machine Learning.



## Instrucciones Generales
En este caso, se busca entender el comportamiento y desempeño de diferentes modelos de clasificación sobre este conjunto de datos, para predecir la variable de interés: **OK**, que servirá para predecir en casos futuros, según los datos de contactabilidad de un cliente, si el cliente aceptará o no contratar el depósito a plazo.

La entrega (grupal o individual, según corresponda) se materializa en un informe donde se contestan las preguntas que se indican en las secciones de "Preguntas", más adelante. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases.

Dentro del informe se puede considerar una tabla de datos de ejecuciones comparadas de los modelos y con diferentes condiciones (balance de clases, proporciones de % entrenamiento-evaluación), apoyando las respuestas a las preguntas correspondientes.

La entrega se realiza en forma de un **informe en formato PDF, adjunto por email** utilizando la plantilla de informe que está en http://dcc.rsolver.com/dcc/docs/InformeActividad.docx

El informe en formato PDF debe ser subido por sólo uno de los integrantes a la siguiente URL

http://aiker.resolver.com/aiker/DocUpload.aspx (*)

(*)Si hay problemas en la carga, enviar el PDF a rsandova@ing.puc.cl y cc: ayudante@aiker.ai

## Paso 1: Instación de las librerías de modelos de clasificación

Esto se ejecuta sólo una vez al comienzo de la sesión de cada persona. No se necesita volver a ejecutar con cada nueva prueba del resto de los scripts. Aquí se incluyen liberías para ejecutar todos los modelos.

In [64]:
install.packages('e1071')
install.packages('caret')
install.packages('randomForest')
install.packages('class')
install.packages("nnet")

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)



##Paso 2: Carga y preprocesamiento de los datos ##

A continuación se cargan el conjunto de datos desde la URL de origen. Esta versión preprocesa el dataset, realizando actividades de limpieza de datos, eliminando filas con algún NA y eliminando unas pocas columnas que se determinan como no-relevantes en el desempeño de modelos de clasificación.

Adicionalmente, se realiza un balance entre las clases, reduciendo la cantidad de ejemplos de la clase mayoritaria para aproximarse a la otra.

Se entiende que con estas acciones de preprocesamiento del dataset, ya se llega en mejores condiciones a este nuevo ejercicio.

Este código se puede ejecutar sólo una vez para uso del dataset resultante en los siguientes pasos.

In [ ]:
# Se cargan las librerías

library(randomForest)
library(caret)
library(e1071)
library(nnet)

In [47]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://www.rsolver.com/dcc/docs/bank-additional-full.csv"

# Se declaran los nombres de las columnas
columnas = c("Edad","Ocupación","EstadoCivil","Educación","Default","Hipotecario","Consumo","Contacto","Mes","Día",
             "Duración","NumContactos","DíasAtrás","Previo","ResultadoPrevio",
             "EmpTasaVar", "IPC", "ICC", "Euribor3m", "NumEmpleados", "OK")

# Se cargan datos principales a una estructura o dataset (marketing.data), asignando nombres de atributos a las columnas.
# Nótese que se incluye la conversión de valores "unknown" a "NA" para facilitar la gestión vacíos más adelante.
marketing.data <- read.table(file = theUrlMain, header = TRUE, sep = ";", col.names = columnas, na.strings=c("unknown","NA"))

# Se eliminan aquellos atributos que no aportan en el desempeño de los modelos
# Esto se determinó en un trabajo previo, fuera de esta actividad
marketing.data$Default <- NULL
marketing.data$DíasAtrás <- NULL
marketing.data$Previo <- NULL
marketing.data$Euribor3m <- NULL

# Se eliminan los registros que tienen algún NA (antes: 'unknown')
# Nótese que se hace esta limpieza posterior a la eliminación de columnas,
# logrando conservar la enorme mayoría de los registros.
marketing.clean <- na.omit(marketing.data)
dim(marketing.clean) # Sólo quedan poco más de 38.000 filas (de las 41.000 originales)

# Se muestran las primeras líneas del dataset, incluyendo sólo las columnas que quedaron.
head(marketing.clean, 20)

[1] 38245    17

,Edad,Ocupación,EstadoCivil,Educación,Hipotecario,Consumo,Contacto,Mes,Día,Duración,NumContactos,ResultadoPrevio,EmpTasaVar,IPC,ICC,NumEmpleados,OK
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,nonexistent,1.1,93.994,-36.4,5191,no
2,57,services,married,high.school,no,no,telephone,may,mon,149,1,nonexistent,1.1,93.994,-36.4,5191,no
3,37,services,married,high.school,yes,no,telephone,may,mon,226,1,nonexistent,1.1,93.994,-36.4,5191,no
4,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,nonexistent,1.1,93.994,-36.4,5191,no
5,56,services,married,high.school,no,yes,telephone,may,mon,307,1,nonexistent,1.1,93.994,-36.4,5191,no
6,45,services,married,basic.9y,no,no,telephone,may,mon,198,1,nonexistent,1.1,93.994,-36.4,5191,no
7,59,admin.,married,professional.course,no,no,telephone,may,mon,139,1,nonexistent,1.1,93.994,-36.4,5191,no
9,24,technician,single,professional.course,yes,no,telephone,may,mon,380,1,nonexistent,1.1,93.994,-36.4,5191,no
10,25,services,single,high.school,yes,no,telephone,may,mon,50,1,nonexistent,1.1,93.994,-36.4,5191,no


In [18]:
# Aquí se arman dos subconjuntos con los datos de cada una de las dos clases.
# Se pueden ver los respectivos tamaños al terminar, evidenciando un desbalance.
set.seed(123)
clean.data.YES <- marketing.clean[marketing.clean$OK == 'yes',]
clean.data.NO <- marketing.clean[marketing.clean$OK == 'no',]
cat("Cantidad de ejemplos por clase: YES & NO\n")
dim(clean.data.YES) # Este es el conjunto más pequeño con poco más de 4.000 ejemplos
dim(clean.data.NO)  # Este es mayoritario con casi 34.000 ejemplos, evidenciando desbalance entre clases

# A continuación se realiza un re-balanceo de las clases (random subsampling),
# que consiste en reducir la cantidad de ejemplos de la clase más masiva, para acercarla a la minoritaria.
balance_ratio <- 1.2 # Se elige un balanceo de 20% más de ejemplos de la clase negativa que la positiva

clean.subdata.YES <- clean.data.YES  # No se aplica sample(); se usan todos los ejemplos de la clase YES (que es la que tiene menos ejemplos)
clean.subdata.NO <- clean.data.NO[sample(nrow(clean.data.NO), balance_ratio*dim(clean.data.YES)[1]), ] # Se elige un subconjunto de los NO

# Muestra cantidad de ejemplos contenidos en cada subconjunto
cat("Cantidad de ejemplos por clase luego del balance entre clases: YES & NO\n")
dim(clean.subdata.YES)
dim(clean.subdata.NO)

# Se juntan para el conjunto de referencia, ahora más balanceado
clean.subdata <- rbind(clean.subdata.YES, clean.subdata.NO)

cat("Resumen del dataset ya limpio")
summary(clean.subdata)

Cantidad de ejemplos por clase: YES & NO


[1] 4258   17

[1] 33987    17

Cantidad de ejemplos por clase luego del balance entre clases: YES & NO


[1] 4258   17

[1] 5109   17

Resumen del dataset ya limpio

      Edad       Ocupación         EstadoCivil         Educación        
 Min.   :18.0   Length:9367        Length:9367        Length:9367       
 1st Qu.:32.0   Class :character   Class :character   Class :character  
 Median :37.0   Mode  :character   Mode  :character   Mode  :character  
 Mean   :40.2                                                           
 3rd Qu.:48.0                                                           
 Max.   :98.0                                                           
 Hipotecario          Consumo            Contacto             Mes           
 Length:9367        Length:9367        Length:9367        Length:9367       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                           

## Ejercicio 1: Preparación conjuntos de entrenamiento y evaluación

En este caso, lo importante es considerar que **es posible cambiar la proporción de datos de entrenamiento y test** viendo el efecto que tiene en el desempeño de los modelos (del ejercicio 2: RandomForest, SVM, NB), viendo que alguno o varios de los modelos entregan mejores resultados considerando lo que más puede interesar, entre Accuracy, Sensitivity, Specificity.

**Pregunta 1** (1.5 puntos)

¿Cuál es la proporción entrenamiento/test que logra mejor desempeño y con cuál de los modelos entre RandomForest, SVM, NB?

El objetivo es probar varias combinaciones cambiando la proporción de datos. Preliminarmente 4 combinaciones, desde 60%/40% hasta 90%/10%. Según los resultados de la ejecución de todos los modelos de clasificación más adelante, determinar y explicitar cuál es la proporción que logra mejores resultados o desmpeño de clasificación y cuál es la influencia del cambio de proporción de entrenamiento/test. Particularmente se espera que los alumnos determinen cómo se elige el mejor modelo (comparando Sensitivity, Specificity, Accuracy).

Se pide documentar en una tabla todas las combinaciones, viendo los indicadores de desempeño más relevantes: Accuracy, Sensitivity, Specificity, **determinando cuál combinación da mejores resultados para cuál de los modelos** (RandomForest, NB, SVM), considerando que el desempeño se logra por maximizar el desempeño de la predicción de YES, además de un buen modelo balanceado (accuracy).


In [49]:
# Primero, se saca una copia del dataset para trabajar sin modificar el original
# Esto permite hacer más modificaciones y correr este código varias veces sin alterar clean.subdata
set.seed(123)
working.data <- clean.subdata

# EJERCICIO 1
# Ahora se configuran los conjuntos de entrenamiento y testing en una proporción
# (por ej: 0.70 = 70% para training y el resto para evaluación o testing)
# Se pide probar diferentes combinaciones (60/40, 70/30, 80/20, 90/10)
# hasta determinar cuál es la mejor en cuál de los modelos.
ratio1 = sample(1:nrow(working.data), size = 0.80*nrow(working.data))
training.data1 = working.data[ratio1,]
testing.data1 = working.data[-ratio1,]

# Se comparan los tamaños de ejemplos para entrenamiento y evaluación.
dim(training.data1)
dim(testing.data1)

head(training.data1)

[1] 7493   17

[1] 1874   17

,Edad,Ocupación,EstadoCivil,Educación,Hipotecario,Consumo,Contacto,Mes,Día,Duración,NumContactos,ResultadoPrevio,EmpTasaVar,IPC,ICC,NumEmpleados,OK
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
36743,27,admin.,single,university.degree,no,no,cellular,jun,mon,138,3,nonexistent,-2.9,92.963,-40.8,5076.2,yes
36892,25,student,single,university.degree,no,yes,telephone,jun,thu,280,6,failure,-2.9,92.963,-40.8,5076.2,yes
15801,39,self-employed,married,high.school,no,no,cellular,jul,mon,761,2,nonexistent,1.4,93.918,-42.7,5228.1,no
38405,39,admin.,single,high.school,yes,no,cellular,oct,tue,386,1,nonexistent,-3.4,92.431,-26.9,5017.5,yes
30730,58,admin.,single,university.degree,yes,no,cellular,may,tue,851,1,nonexistent,-1.8,92.893,-46.2,5099.1,yes
10495,31,services,married,high.school,no,no,telephone,jun,tue,37,6,nonexistent,1.4,94.465,-41.8,5228.1,no


## Ejercicio 2: Comparación de desempeño de modelos de clasificación y su explicación

Habiendo definido los conjuntos de entrenamiento y de test, a continuación se ejecutan unos modelos de clasificación: un RandomForest, un Naive Bayes, y un Support Vector Machine. Cada uno obtiene sus resultados, mostrando sus precisiones en desempeño. No es necesario modificar estos bloques de código. Basta con hacer los cambios en la parte del ejercicio 1 (proporción entrenamiento/test) y volver a ejecutar estos modelos para evaluar su desempeño.

Una vez completado el ejercicio 1 anterior (habiendo quedado con una ejecución de mejor desempeño y habiendo realizado la comparación de los indicadores), se pueden contestar las preguntas a continuación, que se centran en interpretar y analizar comparativamente del desempeño de estos modelos.

**Pregunta 2.1** (1 punto)

Viendo que un balance de clases de 1.2 (sólo un 20% más de ejemplos de la clase negativa sobre la positiva), donde reduce notoriamente la cantidad de ejemplos de la clase negativa, ¿por qué considera que se logra esa mejoría, a pesar de eliminar de entrenamiento y evaluación esa cantidad de ejemplos originales? (Justifique con claridad, según lo que se conoce sobre la forma en que se entrenan los modelos).

**Pregunta 2.2** (1 punto)

Habiendo determinado en el ejercicio 1 cuál es el modelo que tiene mejor desempeño entre todos, con una mejor proporción de entrenamiento/test ¿qué características del modelo apoyan su mejor desempeño sobre los otros modelos, aunque la diferencia haya sido menor? (Justifique con claridad, según lo que se conoce sobre las características particulares de los modelos y por qué ese modelo muestra mejor desempeño que los otros).

Recuerde que sólo se comparan los 3 modelos a continuación.


**Random Forest**

In [50]:
# Random Forest
set.seed(123)
RF_model1 <- randomForest(as.factor(OK) ~ ., data=training.data1, method="class")
RF_predict1 <- predict(RF_model1, testing.data1, type = "class")
confusionMatrix(RF_predict1, as.factor(testing.data1$OK), positive = 'yes')

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  891  78
       yes 132 773
                                          
               Accuracy : 0.8879          
                 95% CI : (0.8728, 0.9019)
    No Information Rate : 0.5459          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7752          
                                          
 Mcnemar's Test P-Value : 0.0002548       
                                          
            Sensitivity : 0.9083          
            Specificity : 0.8710          
         Pos Pred Value : 0.8541          
         Neg Pred Value : 0.9195          
             Prevalence : 0.4541          
         Detection Rate : 0.4125          
   Detection Prevalence : 0.4829          
      Balanced Accuracy : 0.8897          
                                          
       'Positive' Class : yes             
                              

**Naive Bayes**

In [54]:
# Naive Bayes
set.seed(123)
NB_model1 <- naiveBayes(as.factor(OK) ~ ., data=training.data1)
NB_predict1 <- predict(NB_model1, testing.data1, type = "class")
confusionMatrix(NB_predict1, as.factor(testing.data1$OK), positive = 'yes')

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  794 146
       yes 229 705
                                         
               Accuracy : 0.7999         
                 95% CI : (0.781, 0.8178)
    No Information Rate : 0.5459         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.5997         
                                         
 Mcnemar's Test P-Value : 2.291e-05      
                                         
            Sensitivity : 0.8284         
            Specificity : 0.7761         
         Pos Pred Value : 0.7548         
         Neg Pred Value : 0.8447         
             Prevalence : 0.4541         
         Detection Rate : 0.3762         
   Detection Prevalence : 0.4984         
      Balanced Accuracy : 0.8023         
                                         
       'Positive' Class : yes            
                                         

**Support Vector Machine**

In [55]:
# Support Vector Machine (NOTA: toma algunos minutos su ejecución)
set.seed(123)
SVM_model1 <- svm(as.factor(OK) ~ ., data = training.data1, cost = 10, scale = FALSE)
SVM_predict1 <- predict(SVM_model1, testing.data1, type = "class")
confusionMatrix(SVM_predict1, as.factor(testing.data1$OK), positive = 'yes')

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  839 144
       yes 184 707
                                         
               Accuracy : 0.825          
                 95% CI : (0.807, 0.8419)
    No Information Rate : 0.5459         
    P-Value [Acc > NIR] : < 2e-16        
                                         
                  Kappa : 0.6484         
                                         
 Mcnemar's Test P-Value : 0.03129        
                                         
            Sensitivity : 0.8308         
            Specificity : 0.8201         
         Pos Pred Value : 0.7935         
         Neg Pred Value : 0.8535         
             Prevalence : 0.4541         
         Detection Rate : 0.3773         
   Detection Prevalence : 0.4755         
      Balanced Accuracy : 0.8255         
                                         
       'Positive' Class : yes            
                                         

In [56]:
# Se realiza la prueba con 90/10
set.seed(123)
ratio2 = sample(1:nrow(working.data), size = 0.90*nrow(working.data))
training.data2 = working.data[ratio2,]
testing.data2 = working.data[-ratio2,]

# Random Forest

RF_model2 <- randomForest(as.factor(OK) ~ ., data=training.data2, method="class")
RF_predict2 <- predict(RF_model2, testing.data2, type = "class")
cat("Random Forest")
confusionMatrix(RF_predict2, as.factor(testing.data2$OK), positive = 'yes')

# Naive Bayes
NB_model2 <- naiveBayes(as.factor(OK) ~ ., data=training.data2)
NB_predict2 <- predict(NB_model2, testing.data2, type = "class")
cat("Naive Bayes")
confusionMatrix(NB_predict2, as.factor(testing.data2$OK), positive = 'yes')


# Support Vector Machine (NOTA: toma algunos minutos su ejecución)
SVM_model2 <- svm(as.factor(OK) ~ ., data = training.data2, cost = 10, scale = FALSE)
SVM_predict2 <- predict(SVM_model2, testing.data2, type = "class")
cat("Support Vector Machine")
confusionMatrix(SVM_predict2, as.factor(testing.data2$OK), positive = 'yes')

Random Forest

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  434  46
       yes  65 392
                                          
               Accuracy : 0.8815          
                 95% CI : (0.8591, 0.9015)
    No Information Rate : 0.5326          
    P-Value [Acc > NIR] : < 2e-16         
                                          
                  Kappa : 0.7627          
                                          
 Mcnemar's Test P-Value : 0.08755         
                                          
            Sensitivity : 0.8950          
            Specificity : 0.8697          
         Pos Pred Value : 0.8578          
         Neg Pred Value : 0.9042          
             Prevalence : 0.4674          
         Detection Rate : 0.4184          
   Detection Prevalence : 0.4877          
      Balanced Accuracy : 0.8824          
                                          
       'Positive' Class : yes             
                              

Naive Bayes

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  379  74
       yes 120 364
                                          
               Accuracy : 0.793           
                 95% CI : (0.7656, 0.8185)
    No Information Rate : 0.5326          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5868          
                                          
 Mcnemar's Test P-Value : 0.001234        
                                          
            Sensitivity : 0.8311          
            Specificity : 0.7595          
         Pos Pred Value : 0.7521          
         Neg Pred Value : 0.8366          
             Prevalence : 0.4674          
         Detection Rate : 0.3885          
   Detection Prevalence : 0.5165          
      Balanced Accuracy : 0.7953          
                                          
       'Positive' Class : yes             
                              

Support Vector Machine

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  411  80
       yes  88 358
                                          
               Accuracy : 0.8207          
                 95% CI : (0.7946, 0.8448)
    No Information Rate : 0.5326          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.6403          
                                          
 Mcnemar's Test P-Value : 0.5892          
                                          
            Sensitivity : 0.8174          
            Specificity : 0.8236          
         Pos Pred Value : 0.8027          
         Neg Pred Value : 0.8371          
             Prevalence : 0.4674          
         Detection Rate : 0.3821          
   Detection Prevalence : 0.4760          
      Balanced Accuracy : 0.8205          
                                          
       'Positive' Class : yes             
                              

In [29]:
# Se realiza la prueba con 70/30
set.seed(123)
ratio3 = sample(1:nrow(working.data), size = 0.70*nrow(working.data))
training.data3 = working.data[ratio3,]
testing.data3 = working.data[-ratio3,]

# Random Forest

RF_model3 <- randomForest(as.factor(OK) ~ ., data=training.data3, method="class")
RF_predict3 <- predict(RF_model3, testing.data3, type = "class")
cat("Random Forest")
confusionMatrix(RF_predict3, as.factor(testing.data3$OK), positive = 'yes')

# Naive Bayes
NB_model3 <- naiveBayes(as.factor(OK) ~ ., data=training.data3)
NB_predict3 <- predict(NB_model3, testing.data3, type = "class")
cat("Naive Bayes")
confusionMatrix(NB_predict3, as.factor(testing.data3$OK), positive = 'yes')


# Support Vector Machine (NOTA: toma algunos minutos su ejecución)
SVM_model3 <- svm(as.factor(OK) ~ ., data = training.data3, cost = 10, scale = FALSE)
SVM_predict3 <- predict(SVM_model3, testing.data3, type = "class")
cat("Support Vector Machine")
confusionMatrix(SVM_predict3, as.factor(testing.data3$OK), positive = 'yes')

Random Forest

Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1338  122
       yes  208 1143
                                          
               Accuracy : 0.8826          
                 95% CI : (0.8701, 0.8943)
    No Information Rate : 0.55            
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7643          
                                          
 Mcnemar's Test P-Value : 2.881e-06       
                                          
            Sensitivity : 0.9036          
            Specificity : 0.8655          
         Pos Pred Value : 0.8460          
         Neg Pred Value : 0.9164          
             Prevalence : 0.4500          
         Detection Rate : 0.4066          
   Detection Prevalence : 0.4806          
      Balanced Accuracy : 0.8845          
                                          
       'Positive' Class : yes             
                        

Naive Bayes

Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1199  216
       yes  347 1049
                                          
               Accuracy : 0.7997          
                 95% CI : (0.7844, 0.8144)
    No Information Rate : 0.55            
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5992          
                                          
 Mcnemar's Test P-Value : 4.281e-08       
                                          
            Sensitivity : 0.8292          
            Specificity : 0.7755          
         Pos Pred Value : 0.7514          
         Neg Pred Value : 0.8473          
             Prevalence : 0.4500          
         Detection Rate : 0.3732          
   Detection Prevalence : 0.4966          
      Balanced Accuracy : 0.8024          
                                          
       'Positive' Class : yes             
                        

Support Vector Machine

Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1235  200
       yes  311 1065
                                          
               Accuracy : 0.8182          
                 95% CI : (0.8035, 0.8323)
    No Information Rate : 0.55            
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.6357          
                                          
 Mcnemar's Test P-Value : 1.138e-06       
                                          
            Sensitivity : 0.8419          
            Specificity : 0.7988          
         Pos Pred Value : 0.7740          
         Neg Pred Value : 0.8606          
             Prevalence : 0.4500          
         Detection Rate : 0.3789          
   Detection Prevalence : 0.4895          
      Balanced Accuracy : 0.8204          
                                          
       'Positive' Class : yes             
                        

In [30]:
# Se realiza la prueba con 60/40
set.seed(123)
ratio4 = sample(1:nrow(working.data), size = 0.60*nrow(working.data))
training.data4 = working.data[ratio4,]
testing.data4 = working.data[-ratio4,]

# Random Forest

RF_model4 <- randomForest(as.factor(OK) ~ ., data=training.data4, method="class")
RF_predict4 <- predict(RF_model4, testing.data4, type = "class")
cat("Random Forest")
confusionMatrix(RF_predict4, as.factor(testing.data4$OK), positive = 'yes')

# Naive Bayes
NB_model4 <- naiveBayes(as.factor(OK) ~ ., data=training.data4)
NB_predict4 <- predict(NB_model4, testing.data4, type = "class")
cat("Naive Bayes")
confusionMatrix(NB_predict4, as.factor(testing.data4$OK), positive = 'yes')


# Support Vector Machine (NOTA: toma algunos minutos su ejecución)
SVM_model4 <- svm(as.factor(OK) ~ ., data = training.data4, cost = 10, scale = FALSE)
SVM_predict4 <- predict(SVM_model4, testing.data4, type = "class")
cat("Support Vector Machine")
confusionMatrix(SVM_predict4, as.factor(testing.data4$OK), positive = 'yes')

Random Forest

Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1792  158
       yes  276 1521
                                          
               Accuracy : 0.8842          
                 95% CI : (0.8735, 0.8943)
    No Information Rate : 0.5519          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7674          
                                          
 Mcnemar's Test P-Value : 1.952e-08       
                                          
            Sensitivity : 0.9059          
            Specificity : 0.8665          
         Pos Pred Value : 0.8464          
         Neg Pred Value : 0.9190          
             Prevalence : 0.4481          
         Detection Rate : 0.4059          
   Detection Prevalence : 0.4796          
      Balanced Accuracy : 0.8862          
                                          
       'Positive' Class : yes             
                        

Naive Bayes

Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1598  267
       yes  470 1412
                                          
               Accuracy : 0.8033          
                 95% CI : (0.7902, 0.8159)
    No Information Rate : 0.5519          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.6068          
                                          
 Mcnemar's Test P-Value : 1.001e-13       
                                          
            Sensitivity : 0.8410          
            Specificity : 0.7727          
         Pos Pred Value : 0.7503          
         Neg Pred Value : 0.8568          
             Prevalence : 0.4481          
         Detection Rate : 0.3768          
   Detection Prevalence : 0.5023          
      Balanced Accuracy : 0.8069          
                                          
       'Positive' Class : yes             
                        

Support Vector Machine

Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1650  257
       yes  418 1422
                                         
               Accuracy : 0.8199         
                 95% CI : (0.8072, 0.832)
    No Information Rate : 0.5519         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.639          
                                         
 Mcnemar's Test P-Value : 7.348e-10      
                                         
            Sensitivity : 0.8469         
            Specificity : 0.7979         
         Pos Pred Value : 0.7728         
         Neg Pred Value : 0.8652         
             Prevalence : 0.4481         
         Detection Rate : 0.3795         
   Detection Prevalence : 0.4911         
      Balanced Accuracy : 0.8224         
                                         
       'Positive' Class : yes            
                                         

# Ejercicio 3: Implementación de una Red Neuronal
A continuación se declara, entrena y evalúa un modelo de Red Neuronal. Esta primera declaración viene con una configuración inicial, que se podrá modificar para ver posibles mejoras en el desempeño de esta red.

Esta configuración considera lo siguiente:

*     **Nótese que sólo se utilizan algunos atributos del dataset**, que vienen en la declaración de la fórmula (1er argumento) de nnet(). Se pueden eliminar algunos y ver si mejora el desempeño.

*     Se usa una única capa escondida o intermedia. Su cantidad de nodos está dada por el atributo 'size'. Se puede agrandar o reducir para ver posibles mejoras.

*     La cantidad de iteraciones para mejorar el entrenamiento se da por el atributo 'maxit'. Se puede aumentar, esperando mejorar el desempeño.

*     El atributo 'maxNWts' limita el tamaño interno de la red, que dadas las restricciones de capacidad de procesamiento que entrega Google Colab, conviene acotarlo, para evitar sobrepasar la memora y tener una ejecución fallida. No es necesario modificar este atributo.

Hay otros atributos posibles de analizar y modificar en https://www.rdocumentation.org/packages/nnet/versions/7.3-14/topics/nnet. Nótese que la configuración por defecto usa una activación logística, pero es posible aplicar softmax o linout, pero eso requiere de parámetros adicionales.

Ojo/recomendación: dada la naturaleza aleatoria del comportamiento del entrenamiento, en ocasiones la red neuronal no entrega resultados para la clase menos representada y genera un error. En cuyo caso, sólo basta con volver a ejecutar el código, para que - aleatoriamente - logre dar resultados en dicha clase.

**Ejercicio 3:**

Probar diferentes versiones del modelo, cambiando:
*     Los atributos considerados. Por simplicidad se recomienda sólo eliminar algunos de la lista original, para ver si en alguna ejecución esa eliminación genera mejores resultados.
*     La cantidad de nodos de la capa escondida (size).
*     La cantidad de iteraciones (maxit).

Por simplicidad de este ejercicio, se recomienda sólo probar 4 combinaciones de cada uno de los 3 elementos a cambiar. Se pueden elegir los valores de esos cambios y documentar en una tabla de ejecuciones comparadas para contestar la pregunta 3.1.

**Preg 3.1** (1.8 puntos): ¿Cuáles son los parámetros de ejecución del modelo que dan el mejor desempeño de la Red Neuronal?

**Preg 3.2** (0.7 puntos): ¿Logra superar al mejor modelo de los primeros 3 modelos? ¿Por qué considera que si o no y qué caracteristica distinta entre estos 2 modelos hace la diferencia? (En cualquier caso, se pide una posible y teórica explicación de por qué es mejor/peor que ese otro modelo.)

In [95]:
# Combinaciones de atributos
atributos1 = as.factor(OK) ~ Edad + Ocupación + EstadoCivil + Educación + Duración + NumContactos + EmpTasaVar + NumEmpleados
atributos2 = NULL
atributos3 = NULL
atributos4 = NULL
atributos = c(atributos1, atributos2, atributos3, atributos4)

# Combinaciones de cantidad de nodos de la capa intermedia
nodos1 = 25
nodos2 = NULL
nodos3 = NULL
nodos4 = NULL
nodos = c(nodos1, nodos2, nodos3, nodos4)

# Combinaciones de cantidad de iteraciones
iteraciones1 = 3000
iteraciones2 = NULL
iteraciones3 = NULL
iteraciones4 = NULL
iteraciones = c(iteraciones1, iteraciones2, iteraciones3, iteraciones4)

In [99]:
# Red Neuronal
# Lista original de atributos: Edad+Ocupación+EstadoCivil+Educación+Duración+NumContactos+EmpTasaVar+NumEmpleados
set.seed(123)
NN_model <- nnet(
    atributos1,
    data = training.data, MaxNWts = 10000,
    size = nodos1, maxit = iteraciones1 # Combinación 1
)
NN_predict <- predict(NN_model, testing.data, type="class")

# weights:  626
initial  value 5261.260805 
iter  10 value 4304.832801
iter  20 value 4012.487322
iter  30 value 3341.109773
iter  40 value 2944.893959
iter  50 value 2887.548570
iter  60 value 2852.089537
iter  70 value 2699.855088
iter  80 value 2529.102310
iter  90 value 2499.641387
iter 100 value 2495.193127
iter 110 value 2492.793068
final  value 2492.633366 
converged


In [100]:
# A continuación se muestra el resultado de evaluación
cat("Resultados Red Neuronal\n")
confTable <- table("Predicción" = NN_predict, "Observación" = testing.data$OK)
confTable

accuracy <- (confTable[1,1] + confTable[2,2]) / dim(testing.data)[1]
cat("\nAccuracy:    ", accuracy)

sensitivity <- confTable[1,1] / (confTable[1, 1] + confTable[2, 1])
cat("\nSensitivity: ", sensitivity)

specificity <- confTable[2,2] / (confTable[1, 2] + confTable[2, 2])
cat("\nSpecificity: ", specificity)

Resultados Red Neuronal


          Observación
Predicción  no yes
       no  843  94
       yes 180 757


Accuracy:     0.8537887
Sensitivity:  0.8240469
Specificity:  0.8895417

In [78]:
temporal <- confusionMatrix(data = as.factor(NN_predict), reference = as.factor(testing.data$OK), positive = 'no')
temporal

Confusion Matrix and Statistics

          Reference
Prediction  no yes
       no  843  94
       yes 180 757
                                         
               Accuracy : 0.8538         
                 95% CI : (0.837, 0.8695)
    No Information Rate : 0.5459         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.7076         
                                         
 Mcnemar's Test P-Value : 2.821e-07      
                                         
            Sensitivity : 0.8240         
            Specificity : 0.8895         
         Pos Pred Value : 0.8997         
         Neg Pred Value : 0.8079         
             Prevalence : 0.5459         
         Detection Rate : 0.4498         
   Detection Prevalence : 0.5000         
      Balanced Accuracy : 0.8568         
                                         
       'Positive' Class : no             
                                         

In [110]:
set.seed(123)
hora_inicio = Sys.time()
datos_entrenamiento <- training.data
datos_validacion <- testing.data
contador = 1
rows = length(atributos) * length(nodos) * length(iteraciones)

dataframe_final <- data.frame(
    Modelos = Na * 1:rows,
    Atributos = NA * 1:rows,
    Nodos = NA * 1:rows,
    Iteraciones = NA * 1:rows,
    Exactitud = NA * 1:rows,
    Sensibilidad = NA * 1:rows,
    Especificidad= NA * 1:rows
)

for(atributo in seq_along(atributos)) {
    for(nodo in seq_along(nodos)) {
        for(iteracion in seq_along(iteraciones)) {
            cat(paste0("\nModelo ", contador, ", atributo: ", atributo, ", nodo: ", nodo, ", iteración: ", iteracion, "\n"))
            modelo_RN <- nnet(
                atributos[[atributo]], data = datos_entrenamiento, MaxNWts = 10000,
                size = nodos[[nodo]], maxit = iteraciones[[iteracion]]
            )
            prediccion_RN <- predict(modelo_RN, datos_validacion, type="class")
            matriz <- confusionMatrix(data = as.factor(prediccion_RN), reference = as.factor(datos_validacion$OK), positive = "no")

            dataframe_final[contador, 1] <- paste("Modelo 1")
            dataframe_final[contador, 2] <- paste0("atributo", atributo)
            dataframe_final[contador, 3] <- paste0("nodo", nodo)
            dataframe_final[contador, 4] <- paste0("iteracion", iteracion)
            dataframe_final[contador, 5] <- round(matriz$overall[1], 4)
            dataframe_final[contador, 6] <- round(matriz$byClass[1], 4)
            dataframe_final[contador, 7] <- round(matriz$byClass[2], 4)
            contador <- contador + 1
        }
    }
}
hora_final = Sys.time()
hora_inicio - hora_final


Modelo 1, atributo: 1, nodo1, iteración1
# weights:  626
initial  value 5292.474008 
iter  10 value 4223.174808
iter  20 value 4040.085690
iter  30 value 3932.068118
iter  40 value 3832.260952
iter  50 value 3102.911532
iter  60 value 2813.104534
iter  70 value 2767.175160
iter  80 value 2730.504438
iter  90 value 2694.470185
iter 100 value 2611.876883
iter 110 value 2540.854909
iter 120 value 2497.574606
iter 130 value 2473.409718
iter 140 value 2468.934516
iter 150 value 2465.448291
iter 160 value 2459.802794
iter 170 value 2456.952263
iter 180 value 2456.659935
final  value 2456.653236 
converged

Modelo 2, atributo: 1, nodo1, iteración2
# weights:  626
initial  value 5896.075900 
iter  10 value 4430.662887
iter  20 value 4318.441126
iter  30 value 4289.174796
iter  40 value 4103.300544
iter  50 value 3640.425461
iter  60 value 3015.253868
iter  70 value 2888.513721
iter  80 value 2874.572404
iter  90 value 2761.323883
iter 100 value 2569.955689
iter 110 value 2506.155324
iter 120 

Warning message in confusionMatrix.default(data = as.factor(prediccion_RN), reference = as.factor(datos_validacion$OK), :
“Levels are not in the same order for reference and data. Refactoring data to match.”



Modelo 12, atributo: 1, nodo3, iteración4
# weights:  876
initial  value 5276.154808 
iter  10 value 4688.844743
iter  20 value 4023.171916
iter  30 value 3793.536135
iter  40 value 3512.589769
iter  50 value 2899.712246
iter  60 value 2796.800693
iter  70 value 2749.852970
iter  80 value 2639.027468
iter  90 value 2563.587622
iter 100 value 2518.662907
iter 110 value 2477.992819
iter 120 value 2458.185888
iter 130 value 2450.704476
iter 140 value 2445.525733
iter 150 value 2430.813591
iter 160 value 2422.702420
iter 170 value 2418.417083
iter 180 value 2413.783590
iter 190 value 2409.923317
iter 200 value 2407.704862
iter 210 value 2406.110223
iter 220 value 2405.576932
iter 230 value 2405.118956
iter 240 value 2404.260403
iter 250 value 2403.528844
iter 260 value 2402.492052
iter 270 value 2401.446032
iter 280 value 2400.734813
iter 290 value 2399.851977
iter 300 value 2399.423628
iter 310 value 2398.892959
iter 320 value 2398.241303
iter 330 value 2397.418955
iter 340 value 2396.84

Warning message in confusionMatrix.default(data = as.factor(prediccion_RN), reference = as.factor(datos_validacion$OK), :
“Levels are not in the same order for reference and data. Refactoring data to match.”



Modelo 18, atributo: 2, nodo1, iteración2
# weights:  576
initial  value 6926.033548 
final  value 5162.944800 
converged


...


Modelo 31, atributo: 2, nodo4, iteración3
# weights:  921
initial  value 8632.767730 
final  value 5162.944800 
converged


Warning message in confusionMatrix.default(data = as.factor(prediccion_RN), reference = as.factor(datos_validacion$OK), :
“Levels are not in the same order for reference and data. Refactoring data to match.”



Modelo 32, atributo: 2, nodo4, iteración4
# weights:  921
initial  value 7384.856490 
final  value 5162.944800 
converged


Warning message in confusionMatrix.default(data = as.factor(prediccion_RN), reference = as.factor(datos_validacion$OK), :
“Levels are not in the same order for reference and data. Refactoring data to match.”



Modelo 33, atributo: 3, nodo1, iteración1
# weights:  426
initial  value 5762.452253 
iter  10 value 4549.628736
iter  20 value 3497.119876
iter  30 value 3330.386893
iter  40 value 2963.091032
iter  50 value 2920.989971
iter  60 value 2866.522524
iter  70 value 2647.645767
iter  80 value 2549.935788
iter  90 value 2512.528942
iter 100 value 2511.429112
final  value 2511.133587 
converged

Modelo 34, atributo: 3, nodo1, iteración2
# weights:  426
initial  value 9498.878471 
iter  10 value 4309.083332
iter  20 value 3151.187265
iter  30 value 2982.507254
iter  40 value 2954.172402
iter  50 value 2946.761854
iter  60 value 2935.504941
iter  70 value 2913.887041
iter  80 value 2849.971285
iter  90 value 2632.379382
iter 100 value 2525.826942
iter 110 value 2511.754100
iter 120 value 2509.813171
final  value 2509.777043 
converged

Modelo 35, atributo: 3, nodo1, iteración3
# weights:  426
initial  value 6182.389153 
iter  10 value 4517.128206
iter  20 value 4084.803973
iter  30 value 3132

In [141]:
dataframe_final[dataframe_final$Sensibilidad == 1 | dataframe_final$Especificidad == 1, ]

,Atributos,Nodos,Iteraciones,Exactitud,Sensibilidad,Especificidad
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
11,atributo1,nodo3,iteracion3,0.5459,1,0
17,atributo2,nodo1,iteracion1,0.5459,1,0
18,atributo2,nodo1,iteracion2,0.5459,1,0
19,atributo2,nodo1,iteracion3,0.5459,1,0
20,atributo2,nodo1,iteracion4,0.5459,1,0
21,atributo2,nodo2,iteracion1,0.5459,1,0
22,atributo2,nodo2,iteracion2,0.5459,1,0
23,atributo2,nodo2,iteracion3,0.5459,1,0
24,atributo2,nodo2,iteracion4,0.5459,1,0


In [144]:
# Análisis de modelos raros

modelo_RN_raro <- nnet(
    atributos[[2]], data = datos_entrenamiento, MaxNWts = 10000,
    size = nodos[[3]], maxit = iteraciones[[3]]
)
prediccion_RN_rara <- predict(modelo_RN_raro, datos_validacion, type="class")
cat("Niveles de la predicción:", levels(as.factor(prediccion_RN_rara)), "\n")
cat("Niveles de la observación:", levels(as.factor(datos_validacion$OK)), "\n")
matriz_rara <- confusionMatrix(data = as.factor(prediccion_RN_rara), reference = as.factor(datos_validacion$OK), positive = "no")
matriz_rara

# weights:  806
initial  value 5953.492023 
final  value 5162.944800 
converged
Niveles de la predicción: no 
Niveles de la observación: no yes 


Warning message in confusionMatrix.default(data = as.factor(prediccion_RN_rara), :
“Levels are not in the same order for reference and data. Refactoring data to match.”


Confusion Matrix and Statistics

          Reference
Prediction   no  yes
       no  1023  851
       yes    0    0
                                         
               Accuracy : 0.5459         
                 95% CI : (0.523, 0.5686)
    No Information Rate : 0.5459         
    P-Value [Acc > NIR] : 0.5095         
                                         
                  Kappa : 0              
                                         
 Mcnemar's Test P-Value : <2e-16         
                                         
            Sensitivity : 1.0000         
            Specificity : 0.0000         
         Pos Pred Value : 0.5459         
         Neg Pred Value :    NaN         
             Prevalence : 0.5459         
         Detection Rate : 0.5459         
   Detection Prevalence : 1.0000         
      Balanced Accuracy : 0.5000         
                                         
       'Positive' Class : no             
                                         


---

**Complemento: Ejercicio de comprobación manual**


Para verificar que alguno de los modelos realmente predice correctamente, se comprueba con los datos de una persona en particular, pidiendo la predicción al modelo. A continuación hay dos ejemplos, que se pueden modificar para ver su resultado, cambiando valores y también, cambiando el modelo a utilizar en la predicción. No se necesita modificar, ni comentar esta parte en la entrega, sino que se entrega como complemento para quienes tengan el interés de ver cómo se aplica un modelo entrenado en un contexto práctico (en producción)

In [24]:
# Ejemplo 1: La predicción debería ser "YES"
set.seed(123)
sample_x <- clean.subdata[1,]
sample_x[1,1] <- 32       # Edad
sample_x[1,2] <- 'admin.' # Ocupación
sample_x[1,3] <- 'single' # EstadoCivil
sample_x[1,4] <- 'university.degree'  # Educación
sample_x

prediction <- predict(RF_model, sample_x)
prediction


# Ejemplo 2: La predicción debería ser "NO"
sample_x2 <- clean.subdata[1,]
sample_x2[1,10] <- 10   # Duración
sample_x2

prediction <- predict(RF_model, sample_x2)
prediction


,Edad,Ocupación,EstadoCivil,Educación,Hipotecario,Consumo,Contacto,Mes,Día,Duración,NumContactos,ResultadoPrevio,EmpTasaVar,IPC,ICC,NumEmpleados,OK
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
76,32,admin.,single,university.degree,yes,no,telephone,may,mon,1575,1,nonexistent,1.1,93.994,-36.4,5191,yes


76 
yes 
Levels: no yes

,Edad,Ocupación,EstadoCivil,Educación,Hipotecario,Consumo,Contacto,Mes,Día,Duración,NumContactos,ResultadoPrevio,EmpTasaVar,IPC,ICC,NumEmpleados,OK
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
76,41,blue-collar,divorced,basic.4y,yes,no,telephone,may,mon,10,1,nonexistent,1.1,93.994,-36.4,5191,yes


76 
no 
Levels: no yes